# Análise do risco de inadimplência dos mutuários

Este projeto tem o objetivo de preparar um relatório para a divisão de empréstimos de um banco. 

Como tarefa, preciso descobrir quais fatores têm impacto sobre se ele deixará de pagar um empréstimo. O banco já tem alguns dados sobre a capacidade de crédito dos clientes.

O relatório será considerado ao criar uma **pontuação de crédito** de um cliente em potencial. A **pontuação de crédito** é usada para avaliar a capacidade de um devedor em potencial de pagar seu empréstimo.


## Iniciando o projeto

Os passos iniciais de qualquer análise de dados.

In [1]:
# Carregando todas as bibliotecas
import pandas as pd


# Carregando os dados
df = pd.read_csv('/datasets/credit_scoring_eng.csv')

# Exibindo as primeiras 10 linhas
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


## Explorando os dados

**Descrição dos dados**
- `children` - o número de crianças na família
- `days_employed` - experiência de trabalho em dias
- `dob_years` - idade do cliente em anos
- `education` - educação do cliente
- `education_id` - identificador de educação
- `family_status` - estado civil do cliente
- `family_status_id` - identificador de estado civil
- `gender` - gênero do cliente
- `income_type` - tipo de emprego
- `debt` - havia alguma dívida no pagamento do empréstimo
- `total_income` - renda mensal
- `purpose` - o objetivo de obter um empréstimo

In [2]:
# Vamos ver quantas linhas e colunas o conjunto de dados tem
print(f' O número de linhas é {df.shape[0]} e o número de colunas é {df.shape[1]}.')

 O número de linhas é 21525 e o número de colunas é 12.


In [3]:
# Vamos ver as informações gerais sobre os dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# Exibindo as primeiras 10 linhas
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


Logo nesse começo, é possível perceber 2.174 valores ausentes nas colunas "days_employed" e "total_income", assim como alguns dados parecem não estar certos: Há uma quantidade negativa de dias trabalhados em várias linhas, e há uma diferença de grafia dos dados na coluna "education" que podemos investigar.

In [5]:
# Obter informações sobre dados
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Ao aplicar o método "describe" e "info" à planilha, é possível notar alguns valores estranhos:
- Temos um número negativo de crianças na coluna "children", assim como o máximo de crianças é "20", um número possível, porém improvável de filhos
- Temos números negativos de dias trabalhados na coluna "days_employed"
- Há valores ausentes nas colunas "days_employed" e "total_income"

In [6]:
# Verificando a quantidade de valores ausentes na base de dados
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
# Vejamos a tabela filtrada com valores ausentes na primeira coluna com dados ausentes
days_employed_null = df.days_employed.isnull()
df[days_employed_null].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding


Analisando a tabela filtrada com os valores ausentes em "days_employed", parece que para cada linha, os valores de "total_income" também estão ausentes. Vamos verificar.

In [8]:
# Verificando as informações do DataFrame filtrado e se as colunas "days_employed" e "total_income" possuem a mesma quantidade de valores não nulos
df[days_employed_null].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


In [9]:
# Verificando se os dados filtrados nulos de "days_employed" continuam nulos em "total_income"
print(df[days_employed_null]['total_income'])

12      NaN
26      NaN
29      NaN
41      NaN
55      NaN
         ..
21489   NaN
21495   NaN
21497   NaN
21502   NaN
21510   NaN
Name: total_income, Length: 2174, dtype: float64


In [10]:
#Verificando todos os valores únicos filtrados em "total_income"
df[days_employed_null]['total_income'].unique()

array([nan])

**Conclusão intermediária**

O número de linhas na tabela filtrada corresponde ao número de valores ausentes (2174), dessa forma, podemos começar a pensar em como tratar esses dados.

Para cada valor "days_employed" ausente, o valor de "total_income", também está, o que pode indicar que os valores não foram informados ou não puderam ser extraídos, também podendo ser dados correlacionados. 

Vamos verificar a distribuição dos dados e realizar algumas análises.

In [11]:
# Verificando a distribuição de dados ausentes em relação ao conjunto de dados
(df.isna().sum() / df.shape[0]) * 100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

Hmm, parece que os valores ausentes representam cerca de 10% dos dados de "days_employed" e "total_income". Vamos investigar agora quanto à possíveis correlações com outros dados.

In [12]:
# Vamos investigar clientes que não possuem dados sobre as características identificadas e a coluna com os valores ausentes

#Investigando se os dados apresentam informações relevantes quanto à education
print(df[days_employed_null]['education'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à children
print(df[days_employed_null]['children'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à dob_years
print(df[days_employed_null]['dob_years'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à family_status
print(df[days_employed_null]['family_status'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à gender
print(df[days_employed_null]['gender'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à debt
print(df[days_employed_null]['debt'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à purpose
print(df[days_employed_null]['purpose'].value_counts())

secondary education    1408
bachelor's degree       496
SECONDARY EDUCATION      67
Secondary Education      65
some college             55
Bachelor's Degree        25
BACHELOR'S DEGREE        23
primary education        19
Some College              7
SOME COLLEGE              7
Primary Education         1
PRIMARY EDUCATION         1
Name: education, dtype: int64

 0     1439
 1      475
 2      204
 3       36
 20       9
 4        7
-1        3
 5        1
Name: children, dtype: int64

34    69
40    66
31    65
42    65
35    64
36    63
47    59
41    59
30    58
28    57
57    56
58    56
54    55
38    54
56    54
37    53
52    53
39    51
33    51
50    51
51    50
45    50
49    50
29    50
43    50
46    48
55    48
48    46
53    44
44    44
60    39
61    38
62    38
64    37
32    37
27    36
23    36
26    35
59    34
63    29
25    23
24    21
66    20
65    20
21    18
22    17
67    16
0     10
68     9
69     5
20     5
71     5
70     3
72     2
19     1
73     1
Nam

In [13]:
#Verificando a distribuição na base geral

#Investigando se os dados apresentam informações relevantes quanto à education
print(df['education'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à children
print(df['children'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à dob_years
print(df['dob_years'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à family_status
print(df['family_status'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à gender
print(df['gender'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à debt
print(df['debt'].value_counts())

#Investigando se os dados apresentam informações relevantes quanto à income_type
print(df[days_employed_null]['income_type'].value_counts())

print()

#Investigando se os dados apresentam informações relevantes quanto à purpose
print(df['purpose'].value_counts())

secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265


Com essas informações, pudemos notar que:

- A maior parte dos valores ausentes é referente à dados de mulheres casadas, sem filhos, com ensino secundário completo, sem dívidas e com o propósito de casamento. Porém, o restante dos dados parece se mostrar aleatório.


- Comparando a distribuição com os valores de cada coluna na base principal, não parece haver grandes diferenças, indicando que os valores ausentes possivelmente são relacionados apenas entre eles, dias trabalhados e renda total.

**Possíveis motivos para valores ausentes nos dados**
- Como suspeita-se que os valores são aleatórios, a hipótese principal é erro na base de dados, na exportação ou obtenção dos dados.


**Conclusões**

Não foi encontrado nenhum padrão após análise, pois os valores ausentes não parecem estar diretamente ligados a nenhum outro valor de outras colunas, assim como não estão distantes dos valores médios da base não filtrada.

Como os valores ausentes compõem cerca de 10% dos dados de cada coluna, vamos tratá-los e analisar outros possíveis erros na base de dados.

## Transformação de dados

Vamos examinar cada coluna para ver quais problemas podemos ter nelas.

`Coluna Education`

In [14]:
#Corrigindo as grafias na coluna "Education"

#Verificando todos os valores únicos
print(df['education'].unique())

["bachelor's degree" 'secondary education' 'Secondary Education'
 'SECONDARY EDUCATION' "BACHELOR'S DEGREE" 'some college'
 'primary education' "Bachelor's Degree" 'SOME COLLEGE' 'Some College'
 'PRIMARY EDUCATION' 'Primary Education' 'Graduate Degree'
 'GRADUATE DEGREE' 'graduate degree']


In [15]:
#Colocando todos os valores em letras minúscula
df['education']= df['education'].str.lower()

#Verificando se os valores foram alterados
print(df['education'].unique())

["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']


`Coluna Children`

In [16]:
# Vamos ver a distribuição de valores na coluna `children`

#Criando a variável de distribuição de crianças
distribution_children= df['children'].value_counts()

# Imprimindo o resultado da variável criada
distribution_children

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [17]:
# Verificando a porcentagem da distribuição dos dados

#Criando a variável da porcentagem
children_percentage= (distribution_children/distribution_children.sum()) * 100

# Imprimindo o resultado da variável criada
children_percentage

 0     65.732869
 1     22.383275
 2      9.547038
 3      1.533101
 20     0.353078
-1      0.218351
 4      0.190476
 5      0.041812
Name: children, dtype: float64

Estranhamente, há um número negativo (-1) e um número muito alto (20) de filhos nessa coluna.

A hipótese mais provável é que tenha ocorrido um erro de digitação nos dois casos, dessa forma, trocaremos o valor "-1" por "1" e o valor "20" por "2".

In [18]:
# Corrigindo os valores de "-1" filho
df['children'] = df['children'].replace(-1, 1)

# Corrigindo os valores de "20" filhos
df['children'] = df['children'].replace(20, 2)

#Verificando se os valores foram corrigidos
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Pronto! Agora temos uma quantidade de filhos de 0 a 5.

Vamos agora verificar a coluna "days_employed".

`Coluna days_employed`

In [19]:
#Verificando informações de "days_employed"
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Parece que há muitos valores negativos para dias trabalhados... Assim como um valor máximo que é de cerca de 1100 anos. Uma possibilidade é que exista algum erro nas fórmulas que esteja gerando esses valores negativos e muito altos. Vamos primeiro transformar todos os valores em valores positivos.

In [20]:
# Transformando os valores em positivos
df['days_employed'] = df['days_employed'].abs()

In [21]:
# Verificando o resultado
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Os valores agora estão todos positivos! Vamos agora verificar os valores atípicos...

In [22]:
#Verificando o valor máximo de dias trabalhados em anos
print(df['days_employed'].max()/365)

#Verificando a idade máxima na base de dados
df['dob_years'].max()


1100.6997273296713


75

Isso é estranho. A maior quantidade de dias trabalhados na nossa base de dados é de 1100 anos, enquanto a idade máxima de um cliente é de 75 anos. 

Considerando que uma pessoa em média começa a trabalhar com 18 anos, vamos considerar isso em nossos cálculos.

Vamos fazer mais alguns cálculos.

In [23]:
print('Quantidade de dias que uma pessoa de 75 anos viveu:', df['dob_years'].max()*365)
print('Quantidade média de dias que uma pessoa de 75 anos trabalhou:',(75-18)*365)

Quantidade de dias que uma pessoa de 75 anos viveu: 27375
Quantidade média de dias que uma pessoa de 75 anos trabalhou: 20805


Vamos considerar esses cálculos para substituir valores atípicos. Qualquer valor trabalhado maior que `20805` será tratado.

In [24]:
# Verificando a média e mediana dos valores típicos

typical_days_employed = df[df['days_employed'] <= 20805]
median_typical_days_employed = typical_days_employed['days_employed'].median()
mean_typical_days_employed = typical_days_employed['days_employed'].mean()

print(f' A mediana de dias trabalhados é de {median_typical_days_employed} dias ou {median_typical_days_employed/365} anos.')
print(f' A média de dias trabalhados é de {mean_typical_days_employed} dias ou {mean_typical_days_employed/365} anos.')

 A mediana de dias trabalhados é de 1630.0193809778218 dias ou 4.465806523226909 anos.
 A média de dias trabalhados é de 2353.0159319988766 dias ou 6.446618991777744 anos.


In [25]:
# Verificando a média e mediana dos valores atípicos:
atypical_days_employed = df[df['days_employed'] > 20805]
median_atypical_days_employed = atypical_days_employed['days_employed'].median()
mean_atypical_days_employed = atypical_days_employed['days_employed'].mean()

print(f' A mediana de dias trabalhados atípicos é de {median_atypical_days_employed} ou {median_atypical_days_employed/365} anos.')
print(f' A média de dias trabalhados atípicos é de {mean_atypical_days_employed} ou {mean_atypical_days_employed/365} anos.')

 A mediana de dias trabalhados atípicos é de 365213.3062657312 ou 1000.5844007280308 anos.
 A média de dias trabalhados atípicos é de 365004.3099162686 ou 1000.011807989777 anos.


Os valores de médias e medianas entre valores típicos são suficientemente diferentes entre si, portanto, vamos utilizar a mediana para fazer as alterações nos valores atípicos.

In [26]:
df.loc[df['days_employed'] > 20805 , 'days_employed'] = median_typical_days_employed

In [27]:
df['days_employed'].describe()

count    19351.000000
mean      2224.303043
std       2107.307642
min         24.141633
25%        927.009265
50%       1630.019381
75%       2747.423625
max      18388.949901
Name: days_employed, dtype: float64

In [28]:
# Máximo de dias trabalhados na base de dados
print('Em dias:', df['days_employed'].max())
print('Em anos:', df['days_employed'].max()/365)

Em dias: 18388.949900568383
Em anos: 50.38068465909146


`Coluna dob_years`

In [29]:
# Verificando `dob_years` para valores suspeitos
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [30]:
#Analisando os valores ordenados
sorted(df['dob_years'].unique())

[0,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

Parece que há idade zerada na nossa base de dados. Vejamos a porcentagem.

In [31]:
# Verificando a porcentagem da distribuição dos dados de idade

ages= df['dob_years'].value_counts()

#Criando a variável da porcentagem
age_percentage= (ages/ages.sum()) * 100

# Imprimindo o resultado da variável criada
age_percentage.sort_values()

75    0.004646
74    0.027875
73    0.037166
19    0.065041
72    0.153310
20    0.236934
71    0.269454
70    0.301974
69    0.394890
68    0.459930
0     0.469222
21    0.515679
67    0.775842
22    0.850174
66    0.850174
65    0.901278
23    1.180023
24    1.226481
64    1.231127
63    1.249710
62    1.635308
61    1.649245
25    1.658537
60    1.751452
26    1.895470
55    2.058072
59    2.062718
51    2.081301
53    2.132404
57    2.137050
58    2.141696
46    2.206736
54    2.225319
47    2.229965
52    2.248548
56    2.262485
27    2.290360
45    2.308943
28    2.336818
49    2.360046
32    2.369338
43    2.383275
50    2.387921
37    2.494774
48    2.499419
30    2.508711
29    2.531940
44    2.541231
36    2.578397
31    2.601626
39    2.662021
33    2.699187
42    2.773519
38    2.778165
34    2.801394
41    2.819977
40    2.829268
35    2.866434
Name: dob_years, dtype: float64

In [32]:
#Verificando apenas a porcentagem dos valores zerados de idade
print(df[df['dob_years'] == 0]['dob_years'].value_counts()/df['dob_years'].value_counts().sum()*100)

0    0.469222
Name: dob_years, dtype: float64


In [33]:
#Conferindo quantos clientes estão com o valor zerado
df[df['dob_years'] == 0]['dob_years'].count()

101

Apesar de ser uma porcentagem pequena do total, ainda assim são 101 clientes. Vamos verificar a média e mediana dos valores de idade para substituir os zeros.

In [34]:
# Calculando média e mediana para os valores de idade
mean_age= df[df['dob_years'] > 0]['dob_years'].mean()
median_age= df[df['dob_years'] > 0]['dob_years'].median()

print(f' A média de idade é de {mean_age}, enquanto a mediana de idade é de {median_age}')

 A média de idade é de 43.497479462285284, enquanto a mediana de idade é de 43.0


Como não há valores atípicos verificados na base de idades, vamos utilizar a média aproximada para realizar os cálculos.

In [35]:
# Alterando os valores zerados de idade para a média encontrada
df.loc[df['dob_years'] == 0,'dob_years'] = 43

In [36]:
# Verificando se foi corrigido
df[df['dob_years'] == 0]['dob_years'].value_counts()

Series([], Name: dob_years, dtype: int64)

In [37]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

Tudo certo!

`Coluna family_status`

In [38]:
# Vamos ver os valores da coluna
df['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

Não foram localizados valores problemáticos em `family_status`.

`Coluna gender`

In [39]:
# Vamos ver os valores na coluna
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Interessante. Na coluna gênero, há "F", "M" e "XNA". "F" usualmente se refere à "Female" (Mulher), "M" se refere à "Male" (Homem), já "XNA" pode significar um terceiro gênero ou informação incorreta nos dados. Vamos analisar a linha completa.

In [40]:
#Verificando a linha específica
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,some college,2,civil partnership,1,XNA,business,0,32624.825,buy real estate


Analisando a linha com o valor de gênero para XNA, ela não nos diz nada de muito diferente, que poderia indicar algum erro ou sinalização. Nesse caso, vamos considerar como um terceiro gênero e não tratar o valor.

`Coluna income_type`

In [41]:
# Vamos ver os valores na coluna
df['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

Não foram localizados valores problemáticos em `income_type`.

`Duplicatas`

In [42]:
# Verificando se temos dados duplicados
df.duplicated().sum()

71

Bom, parece que temos 71 linhas duplicadas na base de dados.

In [43]:
# Abordando e excluindo as duplicatas
df= df.drop_duplicates().reset_index(drop= True)

In [44]:
# Última verificação se temos duplicatas
df.duplicated().sum()

0

In [45]:
#Verificando o tamanho do conjunto de dados que você tem agora após suas primeiras manipulações com ele
print(f' Temos {df.shape[0]} linhas e {df.shape[1]} colunas na nova base.')

 Temos 21454 linhas e 12 colunas na nova base.


Com a eliminação dos dados duplicados, passamos a ter 71 linhas a menos, resultando em uma base ainda mais adequada para analisar.

# Trabalhando com valores ausentes

Agora, vamos analisar os valores de ID por meio de dicionários, buscando mais agilidade para o tratamento dos dados.

In [46]:
# Encontrando os dicionários (valores com ID)
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [47]:
#Dicionário para educação
education_dict= df[['education', 'education_id']]
education_dict= education_dict.drop_duplicates().reset_index(drop= True)
education_dict

,education,education_id
0,bachelor's degree,0
1,secondary education,1
2,some college,2
3,primary education,3
4,graduate degree,4


In [48]:
#Dicionário para status familiar
family_dict= df[['family_status', 'family_status_id']]
family_dict= family_dict.drop_duplicates().reset_index(drop= True)
family_dict

,family_status,family_status_id
0,married,0
1,civil partnership,1
2,widow / widower,2
3,divorced,3
4,unmarried,4


Agora vamos tratar os valores ausentes.

### Restaurando valores ausentes em `total_income`

Primeiramente, vamos verificar os valores ausentes de receita total, trocando os valores nulos a partir da idade categorizada.

Categorias:
- 'early adulthood'  (20 a 30 anos)
- 'middle adulthood' (31 a 40 anos)
- 'senior adulthood' (41 a 50 anos)
- 'young old'        (51 a 60 anos)
- 'median old'       (61 a 70 anos)
- 'old old'          (71 a 75 anos)

In [49]:
# Vamos escrever uma função que calcule a categoria de idade
# Baseada no artigo: https://www.sciencedirect.com/topics/computer-science/chronological-age#:~:text=Adulthood%20is%20usually%20divided%20into,of%20age%20are%20also%20important.
def age_group(age):
    if age <= 30:
        return 'early adulthood'
    if age <= 40:
        return 'middle adulthood'
    if age <= 50:
        return 'senior adulthood'
    if age <= 60:
        return 'young old'
    if age <= 70:
        return 'median old'
    if age <= 75:
        return 'old old'

In [50]:
# Testando se a função funciona
print(age_group(25))
print(age_group(35))
print(age_group(45))
print(age_group(55))
print(age_group(65))
print(age_group(75))

early adulthood
middle adulthood
senior adulthood
young old
median old
old old


In [51]:
# Criando coluna nova com base na função
df['age_group']= df['dob_years'].apply(age_group)

In [52]:
# Verificar os valores na nova coluna
display(df.head(10)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,senior adulthood
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,middle adulthood
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,middle adulthood
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle adulthood
4,0,1630.019381,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,young old
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,early adulthood
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,senior adulthood
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,senior adulthood
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,middle adulthood
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,senior adulthood


A partir disso, podemos ter como hipóteses de que a renda depende da idade, do grau de escolaridade e do status familiar. Vamos analisar.

Primeiro, criando uma tabela que tenha apenas dados sem valores ausentes. Esses dados serão usados para restaurar os valores ausentes.

In [53]:
# Criando uma tabela sem valores ausentes
df_full = df.dropna()
df_full.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,senior adulthood
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,middle adulthood
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,middle adulthood
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle adulthood
4,0,1630.019381,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,young old
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,early adulthood
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,senior adulthood
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,senior adulthood
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,middle adulthood
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,senior adulthood


In [54]:
# Verificando os valores médios de renda com base no fator grupo de idade
df.groupby(['age_group'])['total_income'].mean().sort_values()

age_group
old old             19575.454327
median old          23245.390243
young old           25482.856294
early adulthood     25817.674826
senior adulthood    28332.806009
middle adulthood    28376.735148
Name: total_income, dtype: float64

In [55]:
# Verificando os valores medianos de renda com base no fator grupo de idade
df.groupby(['age_group'])['total_income'].median().sort_values()

age_group
old old             18611.5935
median old          19705.8550
young old           22056.7710
early adulthood     22957.1850
senior adulthood    24563.6500
middle adulthood    24825.1865
Name: total_income, dtype: float64

A partir desses dados, podemos notar que a idade influencia nos valores de renda, sendo o grupo "middle adulthood" o grupo que tem o valor médio maior. Como o valor de média e mediana são razoavelmente diferentes, melhor usarmos a mediana para eventual substituição.

In [56]:
# Verificando os valores médios de renda com base no fator educação
df.groupby(['education'])['total_income'].mean().sort_values()

education
primary education      21144.882211
secondary education    24594.503037
graduate degree        27960.024667
some college           29045.443644
bachelor's degree      33142.802434
Name: total_income, dtype: float64

In [57]:
# Verificando os valores medianos de renda com base no fator educação
df.groupby(['education'])['total_income'].median().sort_values()

education
primary education      18741.9760
secondary education    21836.5830
graduate degree        25161.5835
some college           25618.4640
bachelor's degree      28054.5310
Name: total_income, dtype: float64

Educação também tem uma influência importante na renda. Vamos levar em consideração.

In [58]:
# Verificando os valores médios de renda com base no fator status familiar
df.groupby(['family_status'])['total_income'].mean().sort_values()

family_status
widow / widower      22984.208556
civil partnership    26694.428597
unmarried            26934.069805
married              27041.784689
divorced             27189.354550
Name: total_income, dtype: float64

In [59]:
# Verificando os valores medianos de renda com base no fator status familiar
df.groupby(['family_status'])['total_income'].median().sort_values()

family_status
widow / widower      20514.190
unmarried            23149.028
civil partnership    23186.534
married              23389.540
divorced             23515.096
Name: total_income, dtype: float64

Ao que parece, o status familiar não é tão relevante para a renda dos clientes. Vamos continuar analisando.

In [60]:
# Verificando os valores médios de renda com base no fator gênero
df.groupby(['gender'])['total_income'].mean().sort_values()

gender
F      24655.604757
M      30907.144369
XNA    32624.825000
Name: total_income, dtype: float64

In [61]:
# Verificando os valores medianos de renda com base no fator gênero
df.groupby(['gender'])['total_income'].median().sort_values()

gender
F      21464.845
M      26834.295
XNA    32624.825
Name: total_income, dtype: float64

Gênero também parece ser um fator relevante para a renda. Novamente, nos ateremos à mediana dos valores.

In [62]:
# Verificando os valores médios de renda com base no fator tipo de renda
df.groupby(['income_type'])['total_income'].mean().sort_values()

income_type
paternity / maternity leave     8612.661000
student                        15712.260000
unemployed                     21014.360500
retiree                        21940.394503
employee                       25820.841683
civil servant                  27343.729582
business                       32386.793835
entrepreneur                   79866.103000
Name: total_income, dtype: float64

In [63]:
# Verificando os valores medianos de renda com base no fator tipo de renda
df.groupby(['income_type'])['total_income'].median().sort_values()

income_type
paternity / maternity leave     8612.6610
student                        15712.2600
retiree                        18962.3180
unemployed                     21014.3605
employee                       22815.1035
civil servant                  24071.6695
business                       27577.2720
entrepreneur                   79866.1030
Name: total_income, dtype: float64

Ao que parece, o tipo de renda também influencia bastante na renda média.

Vamos utilizar os seguintes fatores principais para preenchimento dos valores ausentes, conforme a mediana de cada um, pois houve divergência considerável dos valores entre médias e medianas:
- Grupo de idade
- Educação
- Gênero

In [64]:
#  Escrevendo uma função que usaremos para preencher os valores ausentes
def fill_na_total_income (df, col, parâmetro):
    df_aux = df.groupby([parâmetro])[col].median().reset_index()
    
    for row in range(df_aux.shape[0]):
        grupo = df_aux.loc[row, parâmetro] 
        valor = df_aux.loc[row, col] 
        
        #Preenchendo os valores vazios do grupo com o valor da mediana para esse grupo
        df.loc[df[parâmetro]==grupo, col] = df.loc[df[parâmetro]==grupo, col].fillna(valor)
    
    return df 

In [65]:
#Usando a função
fill_na_total_income (df,'total_income', 'gender')
fill_na_total_income (df,'total_income', 'education')
fill_na_total_income (df,'total_income', 'age_group')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,senior adulthood
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,middle adulthood
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,middle adulthood
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle adulthood
4,0,1630.019381,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,young old
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,senior adulthood
21450,0,1630.019381,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,median old
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,middle adulthood
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,middle adulthood


In [66]:
# Verificando se temos algum erro
print(df['total_income'].isna().sum())

df['total_income'].describe()

0


count     21454.000000
mean      26436.254298
std       15703.063756
min        3306.762000
25%       17219.817250
50%       22710.927000
75%       31330.237250
max      362496.645000
Name: total_income, dtype: float64

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  float64
 11  purpose           21454 non-null  object 
 12  age_group         21454 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


O número total de valores da coluna `total_income` corresponde ao número de valores das outras colunas, não restando valores nulos.

###  Restaurar valores em `days_employed`

Agora, vamos analisar os valores ausentes em `days_employed`, podemos ter como hipóteses de que os dias trabalhados dependem majoritamente da idade.

In [68]:
# Distribuição de `days_employed` medianos com base no grupo de idade
df_full.groupby(['age_group'])['days_employed'].median()

age_group
early adulthood     1046.095163
median old          1630.019381
middle adulthood    1630.019381
old old             1630.019381
senior adulthood    1931.819208
young old           1630.019381
Name: days_employed, dtype: float64

In [69]:
# Distribuição de `days_employed` médios com base no grupo de idade
df_full.groupby(['age_group'])['days_employed'].mean()

age_group
early adulthood     1280.411387
median old          2102.589331
middle adulthood    2087.434418
old old             1912.126700
senior adulthood    2752.828991
young old           2613.158806
Name: days_employed, dtype: float64

In [70]:
# Distribuições estatísticas dos dados de dias trabalhados
df_full['days_employed'].describe()

count    19351.000000
mean      2224.303043
std       2107.307642
min         24.141633
25%        927.009265
50%       1630.019381
75%       2747.423625
max      18388.949901
Name: days_employed, dtype: float64

Os valores de médias e mediandas estão muito diferentes, o que indica valores atípicos nos dados. Baseado nisso, vamos utilizar a mediana.

In [71]:
# Escrevendo a função que calcule a mediana com base nos parâmetros identificados
def fill_na_days_employed (df, col, parâmetro):
    df_aux = df.groupby([parâmetro])[col].median().reset_index()
    
    for row in range(df_aux.shape[0]):
        grupo = df_aux.loc[row, parâmetro] 
        valor = df_aux.loc[row, col] 
        
        #Preenchendo os valores vazios do grupo com o valor da mediana para esse grupo
        df.loc[df[parâmetro]==grupo, col] = df.loc[df[parâmetro]==grupo, col].fillna(valor)
    
    return df 

In [72]:
# Verificando se a função funciona
fill_na_days_employed (df,'days_employed', 'age_group')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,senior adulthood
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,middle adulthood
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,middle adulthood
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle adulthood
4,0,1630.019381,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,young old
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,senior adulthood
21450,0,1630.019381,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,median old
21451,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,middle adulthood
21452,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,middle adulthood


In [73]:
# Verificando se a função funcionou
print(df['days_employed'].isna().sum())

df['days_employed'].describe()

0


count    21454.000000
mean      2163.754556
std       2011.685499
min         24.141633
25%       1023.709707
50%       1630.019381
75%       2523.810846
max      18388.949901
Name: days_employed, dtype: float64

## Categorização de dados

Para responder às perguntas e testar as hipóteses, vamos trabalhar com dados categorizados.

In [74]:
# Verificando os valores para motivação para empréstimo
df['purpose'].value_counts()

wedding ceremony                            791
having a wedding                            768
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
buying property for renting out             651
transactions with commercial real estate    650
housing                                     646
purchase of the house                       646
purchase of the house for my family         638
construction of own property                635
property                                    633
transactions with my real estate            627
building a real estate                      624
buy real estate                             621
purchase of my own house                    620
building a property                         619
housing renovation                          607
buy residential real estate                 606
buying my own car                       

In [75]:
# Verificando os valores exclusivos para motivação para empréstimo
df['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Baseado nessas informações, é possível verificar alguns tópicos comuns que foram escritos de formas diferentes: 
- Comprar uma casa
- Realizar um Casamento
- Comprar um carro
- Educação

Vamos então categorizar esses dados.

In [76]:
# Vamos escrever uma função para categorizar os dados com base em tópicos comunss
def purpose_category(row):
    if 'hous' in row['purpose'] or 'prop' in row['purpose'] or 'real est' in row['purpose']:
        return 'house'
    if 'wedd' in row['purpose']:
        return 'wedding'
    if 'car' in row['purpose']:
        return 'car'
    if 'educ' in row['purpose'] or 'uni' in row['purpose']:
        return 'education'     
    
print(purpose_category(df.loc[2]))
print(purpose_category(df.loc[3]))
print(purpose_category(df.loc[4]))
print(purpose_category(df.loc[5]))

house
education
wedding
house


In [77]:
# Criando uma coluna com as categorias e contando os valores
df['purpose_group']= df.apply(purpose_category, axis=1)
df['purpose_group'].value_counts()

house        10811
car           4306
education     4013
wedding       2324
Name: purpose_group, dtype: int64

In [78]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,senior adulthood,house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,middle adulthood,car
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,middle adulthood,house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle adulthood,education
4,0,1630.019381,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,young old,wedding


**Categorizando dados de renda**

In [79]:
#Analisando estatísticas da renda total
df['total_income'].describe()

count     21454.000000
mean      26436.254298
std       15703.063756
min        3306.762000
25%       17219.817250
50%       22710.927000
75%       31330.237250
max      362496.645000
Name: total_income, dtype: float64

In [80]:
# Criando a função para categorização
def income_group(income):
    if  income <= 5000.0000000:
        return 'income level 1'
    if  income <= 20000.000000:
        return 'income level 2'
    if  income <= 30000.000000:
        return 'income level 3'
    if  income <= 50000.000000:
        return 'income level 4'
    if  income <= 100000.645000:
        return 'income level 5' 
    else:
        return 'income level 6'

In [81]:
#Testando a função
print(income_group(3306.762000))
print(income_group(17219.817250))
print(income_group(22710.927000))
print(income_group(31330.237250))
print(income_group(100000.000))
print(income_group(362496.645000))

income level 1
income level 2
income level 3
income level 4
income level 5
income level 6


In [82]:
#Criando uma nova coluna para a categorização
df['income_group']= df['total_income'].apply(income_group)
df['income_group'].value_counts()

income level 3    8166
income level 2    7343
income level 4    4599
income level 5    1221
income level 6      99
income level 1      26
Name: income_group, dtype: int64

In [83]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,purpose_group,income_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,senior adulthood,house,income level 4
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,middle adulthood,car,income level 2
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,middle adulthood,house,income level 3
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,middle adulthood,education,income level 4
4,0,1630.019381,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,young old,wedding,income level 3
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,early adulthood,house,income level 4
6,0,2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions,senior adulthood,house,income level 4
7,0,152.779569,50,secondary education,1,married,0,M,employee,0,21731.829,education,senior adulthood,education,income level 3
8,2,6929.865299,35,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,middle adulthood,wedding,income level 2
9,0,2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,senior adulthood,house,income level 3


## Verificando as Hipóteses


**Existe alguma relação entre ter filhos e pagar um empréstimo em dia?**

In [84]:
# Verificando os dados das crianças e do pagamento em dia por meio de uma tabela dinâmica
pivot_table_children= df.pivot_table( index='children', columns= 'debt', values= 'gender', aggfunc= 'count')
pivot_table_children

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [85]:
# Calculando a taxa de inadimplência com base no número de filhos
debt_children_tax= pivot_table_children[1]/(pivot_table_children[0]+pivot_table_children[1]) *100
debt_children_tax.sort_values()

children
0    7.543822
3    8.181818
1    9.165808
2    9.492481
4    9.756098
5         NaN
dtype: float64

**Conclusão**

- Todos os clientes com 5 filhos estão em dia com os pagamentos.
- A taxa de inadimplência é menor entre os clientes que tem 0 e 3 filhos.
- A taxa de inadimplência é maior entre os clientes que tem 1, 2 e 4 filhos.

Em geral, há uma relação entre ter filhos e pagar um empréstimo em dia, sendo a taxa de inadimplência maior entre adultos com 1 ou mais filhos.

**Existe alguma relação entre o estado civil e o pagamento de um empréstimo no prazo estipulado?**

In [86]:
# Verificando os dados de status da família e do pagamento em dia
pivot_table_family= df.pivot_table( index='family_status', columns= 'debt', values= 'gender', aggfunc= 'count')
pivot_table_family

debt,0,1
family_status,,
civil partnership,3763,388
divorced,1110,85
married,11408,931
unmarried,2536,274
widow / widower,896,63


In [87]:
# Calculando a taxa de inadimplência com base no status da família
debt_family_tax= pivot_table_family[1]/(pivot_table_family[0]+pivot_table_family[1]) *100
debt_family_tax.sort_values()

family_status
widow / widower      6.569343
divorced             7.112971
married              7.545182
civil partnership    9.347145
unmarried            9.750890
dtype: float64

**Conclusão**

- A taxa de inadimplência é menor entre os clientes divorciados e viúvos.
- A taxa de inadimplência é média entre os clientes casados.
- A taxa de inadimplência é maior entre os clientes em união estável e não casados.

Clientes em união estável ou não casados tem uma taxa de inadimplência maior do que os clientes viúvos, divorciados ou casados.

**Existe uma correlação entre o nível de renda e do pagamento em dia?**

In [88]:
# Verificando os dados do nível de renda e do pagamento em dia
pivot_table_income= df.pivot_table( index='income_group', columns= 'debt', values= 'gender', aggfunc= 'count')
pivot_table_income

debt,0,1
income_group,,
income level 1,24,2
income level 2,6737,606
income level 3,7469,697
income level 4,4255,344
income level 5,1135,86
income level 6,93,6


In [89]:
# Calculando a taxa de inadimplência com base no nível de renda
debt_income_tax= pivot_table_income[1]/(pivot_table_income[0]+pivot_table_income[1]) *100
debt_income_tax.sort_values()

income_group
income level 6    6.060606
income level 5    7.043407
income level 4    7.479887
income level 1    7.692308
income level 2    8.252758
income level 3    8.535391
dtype: float64

**Conclusão**

- A taxa de inadimplência é menor entre os clientes com a maior renda.
- A taxa de inadimplência é média entre os clientes com a renda mais baixa.
- A taxa de inadimplência é maior entre os clientes com renda entre 20.000 e 30.000 anuais.

Clientes com uma renda mais baixa possuem uma taxa de inadimplência mais alta do que os que possuem uma renda maior. Em específico, os clientes com renda entre 20.000 e 30.000 anuais possuem uma taxa de inadimplência maior.

**Como a finalidade do crédito afeta a taxa de inadimplência?**

In [90]:
# Verificando os dados do nível de finalidade de crédito e do pagamento em dia
pivot_table_purpose= df.pivot_table( index='purpose_group', columns= 'debt', values= 'gender', aggfunc= 'count')
pivot_table_purpose

debt,0,1
purpose_group,,
car,3903,403
education,3643,370
house,10029,782
wedding,2138,186


In [91]:
# Conferindo os percentuais de inadimplência para cada finalidade de crédito
debt_income_purpose= pivot_table_purpose[1]/(pivot_table_purpose[0]+pivot_table_purpose[1]) *100
debt_income_purpose.sort_values()

purpose_group
house        7.233373
wedding      8.003442
education    9.220035
car          9.359034
dtype: float64

**Conclusão**

- A taxa de inadimplência é menor entre os clientes que pretendem casar e comprar uma casa.
- A taxa de inadimplência é maior entre os clientes que pretendem se educar e comprar um carro.

Quanto à finalidade de crédito, quem possui a finalidade de comprar um carro ou de se educar, possui taxas de inadimplência maiores do que quem busca comprar uma casa ou se casar.

# Conclusão Geral 

Como conclusões, temos que:

- Houve uma etapa extensa de pré-processamento de dados, incluindo:


    - Análise e verificação dos valores ausentes e seus percentuais;


    - Correção de dados duplicados implícitos  na coluna "education" (com diferenciação de maiúsculas e minúsculas como valores únicos);


    - Correção de dados impossíveis e improváveis na coluna "children" : Alteração do valor "-1" para "1" e de "20" para "2" crianças, baseado na hipótese de erro de digitação;


    -  Correção de dados de dias trabalhados na coluna "days_employed" negativos e impossíveis (por possível erro na fórmula utilizada para os cálculos), transformando todos os valores em valores positivos e utilizando a mediana dos valores para substituir valores > que 57 anos trabalhados (considerado como máximo tendo em vista a idade máxima dos clientes na base de dados);


    - Correção de valores de idade zerados na coluna "dob_years", trocando pelo valor da mediana de idade (43 anos);


    -Não foram localizados valores problemáticos em "family_status" , "gender" e "income_type";


    - Exclusão das duplicatas nos dados;


    - Tratamento de valores ausentes em "total_income" e em "days_employed", com a categorização de dados de idade, renda e propósito do empréstimo.


Após esse extensivo tratamento dos dados, pudemos notar que:


- Em geral, há uma relação entre ter filhos e pagar um empréstimo em dia, sendo a taxa de inadimplência maior entre adultos com 1 ou mais filhos.


- Clientes em união estável ou não casados tem uma taxa de inadimplência maior do que os clientes viúvos, divorciados ou casados.


- Clientes com uma renda mais baixa possuem uma taxa de inadimplência mais alta do que os que possuem uma renda maior. Em específico, os clientes com renda entre 20.000 e 30.000 anuais possuem uma taxa de inadimplência maior.


- Quanto à finalidade de crédito, quem possui a finalidade de comprar um carro ou de se educar, possui taxas de inadimplência maiores do que quem busca comprar uma casa ou se casar.